# Notebook za testiranje

U ovom notebooku napravio sam i testirao logiku iza igre kako bi ju mogao implementirati u posebnom .py fileu.

### 1. Importanje modela i potrebnih biblioteka; modeli s kojima sam testirao su zakomentirani, model za koji sam se odlučio nije

In [2]:
from gensim.models import KeyedVectors
import gensim.downloader as api

In [ ]:
#info = api.info()
#api.load('glove-wiki-gigaword-300', return_path=True)

In [ ]:
#model = KeyedVectors.load_word2vec_format('C:\\Users\\ivosusac/gensim-data\\glove-wiki-gigaword-300\\glove-wiki-gigaword-300.gz', binary=False)

In [ ]:
#len(model.key_to_index)

In [3]:
# careful, loada se cijeli model od 2 milijuna+ rijeci, potraje
model = "model1/model.txt"

word_vectors = KeyedVectors.load_word2vec_format(model, binary=False, limit=10000)

word_vectors.save('model.wordvectors')


In [4]:
#izrazi s dvije rijeci
for key, value in word_vectors.key_to_index.items():
    for letter in key:
        if letter == ':':
            print(key)
            break

New::York_PROPN
United::States_PROPN
per::cent_NOUN
Associated::Press_PROPN
Los::Angeles_PROPN
health::care_NOUN
looking::statements_VERB
White::House_PROPN
High::School_PROPN
e::mail_NOUN
vice::president_NOUN
World::Cup_PROPN
Web::site_NOUN
chief::executive_NOUN
New::Zealand_PROPN
Supreme::Court_PROPN
real::estate_NOUN
NEW::YORK_PROPN
San::Francisco_PROPN
South::Africa_PROPN
New::Jersey_PROPN
Wall::Street_PROPN
North::Carolina_PROPN
Middle::East_PROPN
press::release_NOUN
New::Orleans_PROPN
San::Diego_PROPN
interest::rates_NOUN
AP::Photo_PROPN
North::Korea_PROPN
Hong::Kong_PROPN
United::Nations_PROPN
Of::course_NOUN
law::enforcement_NOUN
prime::minister_NOUN
human::rights_NOUN
St.::Louis_PROPN
President::Barack::Obama_NOUN
Las::Vegas_PROPN
Prime::Minister_PROPN
South::Korea_PROPN
natural::gas_NOUN
Vice::President_PROPN
S::&_NOUN
European::Union_PROPN
Super::Bowl_PROPN
Kansas::City_PROPN
All::Star_NOUN
al::Qaida_PROPN
AT::&_NOUN
climate::change_NOUN
Exchange::Commission_PROPN
Washington

Enchant biblioteka - za provjeru je li riječ u engleskom rječniku

In [3]:
import enchant
dict = enchant.Dict("en_US")

In [4]:
word_vectors.key_to_index

{'</s>_NOUN': 0,
 'in_ADP': 1,
 'for_ADP': 2,
 'that_ADP': 3,
 'is_VERB': 4,
 'on_ADP': 5,
 'The_DET': 6,
 'with_ADP': 7,
 'said_VERB': 8,
 'was_VERB': 9,
 'the_DET': 10,
 'at_ADP': 11,
 'not_ADV': 12,
 'as_ADP': 13,
 'it_PRON': 14,
 'be_VERB': 15,
 'from_ADP': 16,
 'by_ADP': 17,
 'are_VERB': 18,
 'I_PRON': 19,
 'have_VERB': 20,
 'he_PRON': 21,
 'will_AUX': 22,
 'has_VERB': 23,
 'his_PRON': 24,
 'an_DET': 25,
 'this_DET': 26,
 'or_CCONJ': 27,
 'their_PRON': 28,
 'who_PRON': 29,
 'they_PRON': 30,
 'but_CCONJ': 31,
 'had_VERB': 32,
 'year_NOUN': 33,
 'were_VERB': 34,
 'we_PRON': 35,
 'more_ADV': 36,
 'up_ADV': 37,
 'been_VERB': 38,
 'you_PRON': 39,
 'its_PRON': 40,
 'one_NUM': 41,
 'about_ADP': 42,
 'would_AUX': 43,
 'which_DET': 44,
 'out_ADP': 45,
 'can_AUX': 46,
 'It_PRON': 47,
 'all_DET': 48,
 'also_ADV': 49,
 'two_NUM': 50,
 'after_ADP': 51,
 'first_ADV': 52,
 'He_PRON': 53,
 'do_VERB': 54,
 'time_NOUN': 55,
 'than_ADP': 56,
 'when_ADV': 57,
 'We_PRON': 58,
 'over_ADP': 59,
 'last_A

### 2. Filtriranje nepotrebnih riječi
Zbog jednostavnosti uzete su samo imenice kao riječi koje se pogađaju, dodatno su filtriranje tako da sadrže 3+ znaka

In [4]:
import re

# regex that checks if the input word contains 3+ letters in utf-8
pattern = r'^[a-z]{3,}$'

daily_words = {}
guessing_words = {}

for key, value in word_vectors.key_to_index.items():
    splitter = key.split('_')
    if dict.check(splitter[0]):
        if splitter[1] == 'NOUN' and re.match(pattern, splitter[0]):
            if daily_words.get(splitter[0]) is not None:
                daily_words[splitter[0]].append(splitter[1])
                guessing_words[splitter[0]].append(splitter[1])
            else:
                daily_words[splitter[0]] = [splitter[1]]
                guessing_words[splitter[0]] = [splitter[1]]
        elif (splitter[1] == 'ADJ' or splitter[1] == 'VERB') and re.match(pattern, splitter[0]):
            if guessing_words.get(splitter[0]) is not None:
                guessing_words[splitter[0]].append(splitter[1])
            else:
                guessing_words[splitter[0]] = [splitter[1]]

In [ ]:
# ne postoji nijedna riječ koja se ponavlja kao imenica i glagol ili sl., pojednostavljava stvari
for key, value in guessing_words.items():
    if len(value) > 1:
        print(key, value)

In [5]:
import random

### 3. Logika iza igre
Izračunavanje udaljenosti kao indeks u polju najsličnijih riječi, sličnije riječi će imati manji indeks.
Također se pripremaju common nouns kao priprema za implementaciju easy i medium levela

In [6]:
def getIdxDistance(word, day_word, similarity_vector):
  if word == day_word:
    return 0
  for idx, i in enumerate(similarity_vector):
    if i[0] == word:
      return idx + 1

In [5]:
word_vectors.similar_by_word("ball_NOUN", topn=len(word_vectors.key_to_index))

[('balls_NOUN', 0.6992626786231995),
 ('downfield_ADV', 0.6390728950500488),
 ('dribble_NOUN', 0.6168773174285889),
 ('puck_NOUN', 0.598172664642334),
 ('lofted_VERB', 0.5933610200881958),
 ('dribbled_VERB', 0.5688434839248657),
 ('bobbled_VERB', 0.568591296672821),
 ('lob_NOUN', 0.5652653574943542),
 ('footwork_NOUN', 0.5582841038703918),
 ('dribbling_VERB', 0.5472685098648071),
 ('defensively_ADV', 0.5454917550086975),
 ('fielders_NOUN', 0.5384922623634338),
 ('midfield_NOUN', 0.5271012783050537),
 ('bunt_NOUN', 0.5223233699798584),
 ('inbounds::pass_NOUN', 0.5216928124427795),
 ('halfcourt_NOUN', 0.5215900540351868),
 ('scrimmage_NOUN', 0.5122807621955872),
 ('Offensively_ADV', 0.5107829570770264),
 ('lineout_NOUN', 0.5069038271903992),
 ('flicked_VERB', 0.5057433247566223),
 ('backhand_NOUN', 0.5039679408073425),
 ('tackler_NOUN', 0.5039458274841309),
 ('Defensively_ADV', 0.501911461353302),
 ('offensively_ADV', 0.5002042055130005),
 ('keeper_NOUN', 0.4991745054721832),
 ('batted_V

In [14]:
common_nouns = []

with open('1000nouns/top-1000-nouns.txt') as f:
    for line in f:
        common_nouns.append(line.strip())

common_nouns[0] #'ï»¿time', treba popraviti

common_nouns[0] = common_nouns[0][3:]

In [19]:
br = 0
common_nouns_filtered = []
for noun in common_nouns:
    if noun in daily_words.keys():
        common_nouns_filtered.append(noun)

In [24]:
def game(day_word, similarity_vector):
    words = []
    while True:
        word = input('Enter a word: ')

        distance = getIdxDistance(word, day_word, similarity_vector)

        if distance == 0:
            print('Congrats, you found the word! Restart for more challenges :)')
            break

        if distance is None:
            print('The word you entered is not in the dictionary. Try again.')
            continue

        words.append('{} -------- {}'.format(word, distance))
        print('{} -------- {}'.format(word, distance))

### 4. Testiranje same igre

In [28]:
level = input('Choose difficulty level (easy/medium/hard): ')


if level == 'easy':
  easy_day_word = random.choice(common_nouns_filtered[:100])
  easy_day_word_with_tag = '_'.join([easy_day_word, 'NOUN'])
  print(easy_day_word)
  similarity_vector = word_vectors.similar_by_word(easy_day_word_with_tag, topn=len(word_vectors.key_to_index))

  similarity_vector_filtered = []

  for word, similarity in similarity_vector:
        if re.match(pattern, word.split('_')[0]):
                similarity_vector_filtered.append((word.split('_')[0], similarity))
  
  game(easy_day_word, similarity_vector_filtered)
  
  
elif level == 'medium':
  medium_day_word = random.choice(common_nouns_filtered[100:])
  medium_day_word_with_tag = '_'.join([medium_day_word, 'NOUN'])
  print(medium_day_word)
  similarity_vector = word_vectors.similar_by_word(medium_day_word_with_tag, topn=len(word_vectors.key_to_index))

  similarity_vector_filtered = []

  for word, similarity in similarity_vector:
        if re.match(pattern, word.split('_')[0]):
                similarity_vector_filtered.append((word.split('_')[0], similarity))
  
  game(medium_day_word, similarity_vector_filtered)


elif level == 'hard':
  hard_day_word = random.choice(list(daily_words.keys()))
  hard_day_word_with_tag = '_'.join([hard_day_word, 'NOUN'])
  print(hard_day_word)
  similarity_vector = word_vectors.similar_by_word(hard_day_word_with_tag, topn=len(word_vectors.key_to_index))
  
  similarity_vector_filtered = []

  for word, similarity in similarity_vector:
        if re.match(pattern, word.split('_')[0]):
                similarity_vector_filtered.append((word.split('_')[0], similarity))

  game(hard_day_word, similarity_vector_filtered)

else:
  print('Invalid input. Type in easy, medium or hard.')


fairways
fairway -------- 1
Congrats, you found the word! Restart for more challenges :)
